In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [0]:
dbutils.secrets.list(scope="testScope")


[SecretMetadata(key='tokyo-olympics-secret')]

In [0]:
secretkey = dbutils.secrets.get(scope="testScope",key="tokyo-olympics-secret")
secretkey

'[REDACTED]'

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "4ea70484-a0ed-4293-932e-0cb82d189cf9",
    "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="testScope", key="tokyo-olympics-secret"),
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/254beea4-dccf-4626-8762-246272629bd5/oauth2/token"
}

dbutils.fs.mount(
    source = "abfss://tokyo-olympics-data-container@tokyoolympicsdatastorage.dfs.core.windows.net/",
    mount_point = "/mnt/tokyoolympics",
    extra_configs = configs
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3885249095096820>, line 9
      1 configs = {
      2     "fs.azure.account.auth.type": "OAuth",
      3     "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   (...)
      6     "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/254beea4-dccf-4626-8762-246272629bd5/oauth2/token"
      7 }
----> 9 dbutils.fs.mount(
     10     source = "abfss://tokyo-olympics-data-container@tokyoolympicsdatastorage.dfs.core.windows.net/",
     11     mount_point = "/mnt/tokyoolympics",
     12     extra_configs = configs
     13 )

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionErro

In [0]:
%fs
ls "mnt/tokyoolympics"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympics/Raw data/,Raw data/,0,1734086412000
dbfs:/mnt/tokyoolympics/Transformed data/,Transformed data/,0,1734086426000


In [0]:
# No need to create Spark session object here manually, Azure databricks will take care of it

spark

In [0]:
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolympics/Raw data/athletes.csv")

coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/Raw data/coaches.csv")

In [0]:
entriesGender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/Raw data/entriesGender.csv")

medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/Raw data/medals.csv")

teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/Raw data/teams.csv")


In [0]:
# Without inferSchema data types of columns might not be correct, You have to do castType transformations

entriesGender = spark.read.format("csv").option("header","true").load("/mnt/tokyoolympics/Raw data/entriesGender.csv")

In [0]:
athletes.show()
coaches.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()
coaches.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesGender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
entriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
entriesGender = entriesGender.withColumn("Female", col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
entriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
teams.printSchema()
teams.show()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|     

In [0]:
# Retrieving list of countries with number of gold medals

medals.orderBy("Gold", ascending=False).show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  14|                Cuba|   7|     3|     5|   15|           18|
|  13|         New Zealand|   7|     6|     7|   20|          

In [0]:
# Retrieving top 5 countries with number of gold medals with only gold column

medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show(5)

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
+--------------------+----+
only showing top 5 rows



In [0]:
# Calculate the average number of entries by gender for each discipline

avg_entries_by_gender = entriesGender.withColumn("Avg_Male", entriesGender["Male"]/entriesGender["Total"]).withColumn("Avg_Female", entriesGender["Female"]/entriesGender["Total"])

avg_entries_by_gender.show(5)

+-------------------+------+----+-----+------------------+------------------+
|         Discipline|Female|Male|Total|          Avg_Male|        Avg_Female|
+-------------------+------+----+-----+------------------+------------------+
|     3x3 Basketball|    32|  32|   64|               0.5|               0.5|
|            Archery|    64|  64|  128|               0.5|               0.5|
|Artistic Gymnastics|    98|  98|  196|               0.5|               0.5|
|  Artistic Swimming|   105|   0|  105|               0.0|               1.0|
|          Athletics|   969|1072| 2041|0.5252327290543851|0.4747672709456149|
+-------------------+------+----+-----+------------------+------------------+
only showing top 5 rows



In [0]:
#Adding additional two columns to the entriesGender Data frame

entriesGender = avg_entries_by_gender
entriesGender.show(5)

+-------------------+------+----+-----+------------------+------------------+
|         Discipline|Female|Male|Total|          Avg_Male|        Avg_Female|
+-------------------+------+----+-----+------------------+------------------+
|     3x3 Basketball|    32|  32|   64|               0.5|               0.5|
|            Archery|    64|  64|  128|               0.5|               0.5|
|Artistic Gymnastics|    98|  98|  196|               0.5|               0.5|
|  Artistic Swimming|   105|   0|  105|               0.0|               1.0|
|          Athletics|   969|1072| 2041|0.5252327290543851|0.4747672709456149|
+-------------------+------+----+-----+------------------+------------------+
only showing top 5 rows



In [0]:
#athletes.repartition(1).write.option("header","true").csv("dbfs:/mnt/tokyoolympics/Transformed data/athletes")

# For overwriting the existing transformed file
athletes.repartition(1).write.option("header","true").mode("overwrite").csv("dbfs:/mnt/tokyoolympics/Transformed data/athletes")

In [0]:
coaches.repartition(1).write.option("header","true").mode("overwrite").csv("dbfs:/mnt/tokyoolympics/Transformed data/coaches")

entriesGender.repartition(1).write.option("header","true").mode("overwrite").csv("dbfs:/mnt/tokyoolympics/Transformed data/entriesGender")

medals.repartition(1).write.option("header","true").mode("overwrite").csv("dbfs:/mnt/tokyoolympics/Transformed data/medals")

teams.repartition(1).write.option("header","true").mode("overwrite").csv("dbfs:/mnt/tokyoolympics/Transformed data/teams")